In [2]:
!pip install tensorly

                                              0.0/7.4 MB ? eta -:--:--
                                              0.1/7.4 MB 871.5 kB/s eta 0:00:09
                                              0.2/7.4 MB 1.5 MB/s eta 0:00:05
     -                                        0.3/7.4 MB 2.3 MB/s eta 0:00:04
     -                                        0.3/7.4 MB 2.3 MB/s eta 0:00:04
     --                                       0.4/7.4 MB 2.0 MB/s eta 0:00:04
     -----                                    1.0/7.4 MB 3.8 MB/s eta 0:00:02
     -------                                  1.4/7.4 MB 4.3 MB/s eta 0:00:02
     --------                                 1.6/7.4 MB 4.3 MB/s eta 0:00:02
     ---------                                1.8/7.4 MB 4.3 MB/s eta 0:00:02
     ----------                               2.0/7.4 MB 4.3 MB/s eta 0:00:02
     -----------                              2.2/7.4 MB 4.4 MB/s eta 0:00:02
     ------------                             2.4/7.4 MB 4.4 MB/s eta


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import random
import time
import torch
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import tensorly as tl
from tensorly.decomposition import tucker
from torch.nn.functional import mse_loss

import math
from torch.optim.optimizer import Optimizer

torch.manual_seed(0)

ModuleNotFoundError: No module named 'tensorly'

In [ ]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100, 200, 150), r=10):
    """
    Создаёт 3D тензор с заданным размером size и рангом r.
    """
    # Создаём случайные матрицы U для каждого измерения
    U = [torch.randn(dim, r).double() for dim in size]

    # Создаём случайное ядро разложения G
    G = torch.randn(r, r, r).double()

    # Восстанавливаем тензор из ядра и матриц
    data = torch.einsum('ijk,ai,bj,ck->abc', G, U[0], U[1], U[2])

    return data, U, G

In [ ]:
# Размерность тензора и ранг
size = (100, 200, 300)
r = 10

# Генерация тензора
data, U, G = get_tensor(size, r)

# Добавление шума
noise = torch.randn_like(data) * 1e-2
data_noisy = data + noise

# Проверяем размеры и выводим
print("Размеры тензора:", data_noisy.shape)
print("Размеры матриц U:", [u.shape for u in U])
print("Размер ядра G:", G.shape)

In [ ]:
def repair_tensor(G, U):
    """
    Восстанавливает тензор по ядру G и матрицам U.

    Args:
        G (torch.Tensor): ядро разложения Таккера
        U (list of torch.Tensor): список факторизационных матриц

    Returns:
        torch.Tensor: восстановленный тензор
    """
    # Последовательно разворачиваем ядро по матрицам U
    data = G
    for mode, u in enumerate(U):
        data = torch.tensordot(data, u, dims=([0], [1]))  # Перемножение вдоль правильных размерностей
    return data.permute(0, 1, 2)  # Возвращаем тензор в правильный порядок осей
